In [ ]:
import polars as pl
import sys
import yaml
import os
import numpy as np
import pandas as pd

In [ ]:
sys.path.append('../src/')
project_root = os.path.dirname(os.getcwd())

In [ ]:
# Directorios para los archivos de parámetros y los datos
parameters_directory = os.path.join(project_root, 'src', 'parameters')
data_raw_directory = os.path.join(project_root, 'data', 'raw')
data_processed_directory = os.path.join(project_root, 'data', 'processed')
data_features_directory = os.path.join(project_root, 'data', 'features')

In [ ]:
# Lista todos los archivos YAML en el directorio especificado
yaml_files = [f for f in os.listdir(parameters_directory) if f.endswith('.yml')]

# Diccionario para guardar los parámetros cargados
parameters = {}

# Carga cada archivo YAML
for yaml_file in yaml_files:
    with open(os.path.join(parameters_directory, yaml_file), 'r') as file:
        data = yaml.safe_load(file)
        key_name = f'parameters_{yaml_file.replace(".yml", "")}'
        parameters[key_name] = data

PROCESSING DATA

In [ ]:
# # Nodos
# from functions.processing import (validate_tags_pl, 
#                              validate_dtypes_pl, 
#                              change_names_pl, 
#                              change_dtype_pl,
#                              delete_accents_pl,
#                              standardize_binary_values_pl,
#                              impute_missing_values_pl)

In [ ]:
# tag_dict_path = os.path.join(data_raw_directory, parameters['parameters_catalog']['tag_dict_path'])
# raw_data_path = os.path.join(data_raw_directory, parameters['parameters_catalog']['raw_data_path'])
# 
# tag_dict = pl.read_excel(tag_dict_path)
# data_raw = pl.read_csv(raw_data_path)

In [ ]:
# prueba1 = validate_tags_pl(data_raw, tag_dict)

In [ ]:
# prueba2 = validate_dtypes_pl(prueba1, tag_dict)

In [ ]:
# prueba3 = change_names_pl(prueba2, tag_dict)

In [ ]:
# prueba4 = change_dtype_pl(prueba3, tag_dict)

In [ ]:
# prueba5 = delete_accents_pl(prueba4)

In [ ]:
# prueba6 = standardize_binary_values_pl(prueba5, parameters['parameters_processing'])

In [ ]:
# prueba7 = impute_missing_values_pl(prueba6)

FEATURING DATA

In [ ]:
# Nodos
from functions.featuring import (new_features_pl,
                                 add_target_variable_pl,
                                 one_hot_encoding_pl,
                                 random_forest_selection_pl,
                                 conditional_entropy_selection_pl,
                                 intersect_top_features_pl)

In [ ]:
processed_data_path = os.path.join(data_processed_directory, parameters['parameters_catalog']['processed_data_path'])

data_processed = pl.read_csv(processed_data_path)

In [ ]:
prueba8 = new_features_pl(data_processed, parameters['parameters_featuring'])

In [ ]:
target_directory = os.path.join(project_root, 'data', 'processed')
target_path = os.path.join(target_directory, parameters['parameters_catalog']['target_column_path'])
target = pl.read_csv(target_path)
prueba9 = add_target_variable_pl(prueba8, target, parameters['parameters_featuring'])

In [ ]:
prueba10 = one_hot_encoding_pl(prueba9, parameters['parameters_featuring'])

In [ ]:
prueba11 = random_forest_selection_pl(prueba10, parameters['parameters_featuring'])

In [ ]:
prueba12 = conditional_entropy_selection_pl(prueba10, parameters['parameters_featuring'])

In [ ]:
prueba13 = intersect_top_features_pl(prueba11, prueba12, prueba10, parameters['parameters_featuring'])